### New Section

In [0]:
!pip install tensorflow-gpu==2.0.0-beta0
!pip install tensorflow_hub

In [0]:
!pip install -U -q PyDrive
!rm -r -f pdd_new
!git clone https://github.com/Kaliostrogoblin/PDD.git

In [0]:
import os
os.chdir('PDD')
# verify if we are in correct directory
os.listdir()

In [0]:

from google.colab import drive

drive.mount('/content/gdrive')




In [0]:
from pdd.utils.data_utils import create_dataset_from_dir
from pdd.datasets.allcrops import load_data

train_data_path, test_data_path = load_data(split_on_train_test=True, random_state=13)

print("Loading datasets...")
train_dataset = create_dataset_from_dir(train_data_path, shuffle=True)
test_dataset = create_dataset_from_dir(test_data_path, shuffle=True)

### Quant model

In [0]:
#Indicate where you keep quantized model in Google Drive
TFLITE_MODEL="/content/gdrive/My Drive/mobilenet_quant_v1_224.tflite"

In [0]:
import tensorflow as tf
# Load TFLite model and see some details about input/output

tflite_interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL)

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

In [0]:
tflite_interpreter.resize_tensor_input(input_details[0]['index'], (len(test_dataset['data']), 256, 256, 3))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (len(test_dataset['data']), 30))
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

In [0]:
tflite_interpreter.set_tensor(input_details[0]['index'], test_dataset['data'].astype('float32'))

tflite_interpreter.invoke()

tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
#tflite_model_predictions = tflite_interpreter.get_tensor(len(test_dataset['target']), 30)
print("Prediction results shape:", tflite_model_predictions.shape)

In [0]:
import numpy as np
from numpy import load

In [0]:
#Saving data about model in .npz format
np.savez("/content/gdrive/My Drive/pdd_model_new/tfmodels/model_quant.npz", tflite_pred=tflite_model_predictions, y_true=test_dataset['target'])
data = load('/content/gdrive/My Drive/pdd_model_new/tfmodels/model_quant.npz')

In [0]:
data['tflite_pred'].argmax(axis=1)
data['tflite_pred'].argmax(axis=1)==data['y_true']
data['tflite_pred']

In [0]:
#Look at confusion matrix of your model
from pdd.utils.metrics_vis import plot_confusion_matrix
plot_confusion_matrix(y_true=test_dataset['target'], 
                      y_pred=data['tflite_pred'].argmax(axis=1),
                      target_names=test_dataset['target_names'],savefig=True)

In [0]:
#Look at incorrect predictions of your model
from pdd.utils.metrics_vis import plot_incorrect_predictions
plot_incorrect_predictions(imgs=test_dataset['data'], 
                           y_true=test_dataset['target'],
                           y_pred=data['tflite_pred'].argmax(axis=1),
                           target_names=test_dataset['target_names'],
                           savefig=True)

In [0]:
#Indicate, where you want to save confusion matrix and incorrect predictions of your model
!cp 'confusion_matrix.png' "/content/gdrive/My Drive/pdd_model_new/tfmodels/model_quant_confusion_matrix.png"
!cp 'incorrect_preds.png' "/content/gdrive/My Drive/pdd_model_new/tfmodels/model_quant_incorrect_preds.png"

### Lite model

In [0]:
#Indicate where you keep non-quantized model in Google Drive
TFLITE_MODEL = "/content/gdrive/My Drive/pdd_model_new/tfmodels/model.tflite"

In [0]:
tflite_interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL)

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

In [0]:
tflite_interpreter.resize_tensor_input(input_details[0]['index'], (len(test_dataset['data']), 256, 256, 3))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (len(test_dataset['data']), 30))
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

In [0]:
tflite_interpreter.set_tensor(input_details[0]['index'], test_dataset['data'].astype('float32'))

tflite_interpreter.invoke()

tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
#tflite_model_predictions = tflite_interpreter.get_tensor(len(test_dataset['target']), 30)
print("Prediction results shape:", tflite_model_predictions.shape)

In [0]:
#Saving data about model in .npz format
np.savez("/content/gdrive/My Drive/pdd_model_new/tfmodels/model.npz", tflite_pred=tflite_model_predictions, y_true=test_dataset['target'])
data = load('/content/gdrive/My Drive/pdd_model_new/tfmodels/model.npz')

In [0]:
data['tflite_pred'].argmax(axis=1)
data['tflite_pred'].argmax(axis=1).size
data['tflite_pred'].argmax(axis=1)==data['y_true']
data['tflite_pred']

In [0]:
#Look at confusion matrix of your model
from pdd.utils.metrics_vis import plot_confusion_matrix
plot_confusion_matrix(y_true=test_dataset['target'], 
                      y_pred=data['tflite_pred'].argmax(axis=1),
                      target_names=test_dataset['target_names'],savefig=True)

In [0]:
#Look at incorrect predictions of your model
from pdd.utils.metrics_vis import plot_incorrect_predictions
plot_incorrect_predictions(imgs=test_dataset['data'], 
                           y_true=test_dataset['target'],
                           y_pred=data['tflite_pred'].argmax(axis=1),
                           target_names=test_dataset['target_names'],
                           savefig=True)

In [0]:
##Indicate, where you want to save confusion matrix and incorrect predictions of your model
!cp 'confusion_matrix.png' "/content/gdrive/My Drive/pdd_model_new/tfmodels/model_confusion_matrix.png"
!cp 'incorrect_preds.png' "/content/gdrive/My Drive/pdd_model_new/tfmodels/model_incorrect_preds.png"